In [1]:
import json 
from evm.tools.fixture_tests import (
    filter_fixtures,
    generate_fixture_tests,
    hash_log_entries,
    load_fixture,
    normalize_statetest_fixture,
    should_run_slow_tests,
)

from evm.vm.forks import (
    TangerineWhistleVM,
    FrontierVM,
    HomesteadVM,
    SpuriousDragonVM,
    ByzantiumVM,
    ShardingVM,
)

from evm.vm.forks.tangerine_whistle.vm_state import TangerineWhistleVMState

from evm.db.state import (
    MainAccountStateDB,
    ShardingAccountStateDB,
)

from trie import (
    BinaryTrie,
    HexaryTrie,
)

from evm.rlp.headers import (
    BlockHeader,
    CollationHeader,
)
from evm.db.chain import ChainDB



from evm.db import (
    get_db_backend,
)

from eth_keys import keys

In [2]:
transaction_string = '''{
    "ContractCreationSpam" : {
        "_info" : {
            "comment" : "",
            "filledwith" : "cpp-1.3.0+commit.6e0ce939.Linux.g++",
            "lllcversion" : "Version: 0.4.18-develop.2017.9.25+commit.a72237f2.Linux.g++",
            "source" : "src/GeneralStateTestsFiller/stAttackTest/ContractCreationSpamFiller.json",
            "sourceHash" : "87301d69f555c6e24cd899f79edd0f7b3d11c395395cc77c63b8ebfb4f737452"
        },
        "env" : {
            "currentCoinbase" : "0x2adc25665018aa1fe0e6bc666dac8fc2697ff9ba",
            "currentDifficulty" : "0x20000",
            "currentGasLimit" : "0x174876e800",
            "currentNumber" : "0x01",
            "currentTimestamp" : "0x03e8",
            "previousHash" : "0x5e20a0453cecd065ea59c37ac63e079ee08998b6045136a8ce6635c7912ec0b6"
        },
        "post" : { 
            "EIP150" : [
                {
                    "hash" : "0x7e7ff82e1c33ffbeb3dedb679498c9716f83c697f497d75aca467ae936b8ae45",
                    "indexes" : {
                        "data" : 0,
                        "gas" : 0,
                        "value" : 0
                    },
                    "logs" : "0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347"
                }
            ] 
        },
        "pre" : {
            "0x6a0a0fc761c612c340a0e98d33b37a75e5268472" : {
                "balance" : "0x0de0b6b3a7640000",
                "code" : "0x7f6004600c60003960046000f3600035ff00000000000000000000000000000000600052602060006000f0600054805b6001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1505a616000106200002f57600055",
                "nonce" : "0x00",
                "storage" : {
                }
            },
            "0xa94f5374fce5edbc8e2a8697c15331677e6ebf0b" : {
                "balance" : "0xe8d4a51000",
                "code" : "",
                "nonce" : "0x00",
                "storage" : {
                }
            }
        },
        "transaction" : {
            "data" : [
                ""
            ],
            "gasLimit" : [
                "0x0c3500"
            ],
            "gasPrice" : "0x01",
            "nonce" : "0x00",
            "secretKey" : "0x45a915e4d060149eb4365960e6a7a45f334393093061116b197e3240065ff2d8",
            "to" : "0x6a0a0fc761c612C340a0e98d33b37a75e5268472",
            "value" : [
                "0x00"
            ]
        }
    }
}'''

transaction_obj = json.loads(transaction_string)
transaction_obj['ContractCreationSpam'] 

{'_info': {'comment': '',
  'filledwith': 'cpp-1.3.0+commit.6e0ce939.Linux.g++',
  'lllcversion': 'Version: 0.4.18-develop.2017.9.25+commit.a72237f2.Linux.g++',
  'source': 'src/GeneralStateTestsFiller/stAttackTest/ContractCreationSpamFiller.json',
  'sourceHash': '87301d69f555c6e24cd899f79edd0f7b3d11c395395cc77c63b8ebfb4f737452'},
 'env': {'currentCoinbase': '0x2adc25665018aa1fe0e6bc666dac8fc2697ff9ba',
  'currentDifficulty': '0x20000',
  'currentGasLimit': '0x174876e800',
  'currentNumber': '0x01',
  'currentTimestamp': '0x03e8',
  'previousHash': '0x5e20a0453cecd065ea59c37ac63e079ee08998b6045136a8ce6635c7912ec0b6'},
 'post': {'EIP150': [{'hash': '0x7e7ff82e1c33ffbeb3dedb679498c9716f83c697f497d75aca467ae936b8ae45',
    'indexes': {'data': 0, 'gas': 0, 'value': 0},
    'logs': '0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'}]},
 'pre': {'0x6a0a0fc761c612c340a0e98d33b37a75e5268472': {'balance': '0x0de0b6b3a7640000',
   'code': '0x7f6004600c60003960046000f3600035ff0

In [3]:
normalize_fixture = normalize_statetest_fixture(transaction_obj['ContractCreationSpam'] , "EIP150", 0)
normalize_fixture

{'env': {'currentCoinbase': b'*\xdc%fP\x18\xaa\x1f\xe0\xe6\xbcfm\xac\x8f\xc2i\x7f\xf9\xba',
  'currentDifficulty': 131072,
  'currentGasLimit': 100000000000,
  'currentNumber': 1,
  'currentTimestamp': 1000,
  'previousHash': b'^ \xa0E<\xec\xd0e\xeaY\xc3z\xc6>\x07\x9e\xe0\x89\x98\xb6\x04Q6\xa8\xcef5\xc7\x91.\xc0\xb6'},
 'post': {'hash': b'~\x7f\xf8.\x1c3\xff\xbe\xb3\xde\xdbg\x94\x98\xc9qo\x83\xc6\x97\xf4\x97\xd7Z\xcaFz\xe96\xb8\xaeE',
  'logs': b'\x1d\xccM\xe8\xde\xc7]z\xab\x85\xb5g\xb6\xcc\xd4\x1a\xd3\x12E\x1b\x94\x8at\x13\xf0\xa1B\xfd@\xd4\x93G'},
 'pre': {b'j\n\x0f\xc7a\xc6\x12\xc3@\xa0\xe9\x8d3\xb3zu\xe5&\x84r': {'balance': 1000000000000000000,
   'code': b'\x7f`\x04`\x0c`\x009`\x04`\x00\xf3`\x005\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`\x00R` `\x00`\x00\xf0`\x00T\x80[`\x01\x01\x80`\x00R`\x00\x80` \x81\x80\x87`\x06\xf1P`\x01\x01\x80`\x00R`\x00\x80` \x81\x80\x87`\x06\xf1P`\x01\x01\x80`\x00R`\x00\x80` \x81\x80\x87`\x06\xf1P`\x01\x01\x80`\x00R`\x00\x80` \x8

In [4]:
def get_block_hash_for_testing(self, block_number):
    if block_number >= self.block_number:
        return b''
    elif block_number < 0:
        return b''
    elif block_number < self.block_number - 256:
        return b''
    else:
        return keccak(text="{0}".format(block_number)) 

In [5]:
def get_prev_hashes_testing(self, last_block_hash, db):
    prev_hashes = []
    return prev_hashes

In [6]:
TangerineWhistleVMStateForTesting = TangerineWhistleVMState.configure(
    __name__='TangerineWhistleVMStateForTesting',
    get_ancestor_hash=get_block_hash_for_testing,
)
TangerineWhistleVMForTesting = TangerineWhistleVM.configure(
    __name__='TangerineWhistleVMForTesting',
    _state_class=TangerineWhistleVMStateForTesting,
    get_prev_hashes=get_prev_hashes_testing,
)

In [7]:
ShardingVMForTesting = ''
def test_state_fixtures(fixture, fixture_vm_class):
    if fixture_vm_class is not ShardingVMForTesting:
        account_state_class = MainAccountStateDB
        trie_class = HexaryTrie
        header = BlockHeader(
            coinbase=fixture['env']['currentCoinbase'],
            difficulty=fixture['env']['currentDifficulty'],
            block_number=fixture['env']['currentNumber'],
            gas_limit=fixture['env']['currentGasLimit'],
            timestamp=fixture['env']['currentTimestamp'],
            parent_hash=fixture['env']['previousHash'],
        )
    else:
        account_state_class = ShardingAccountStateDB
        trie_class = BinaryTrie
        header = CollationHeader(
            shard_id=fixture['env']['shardID'],
            expected_period_number=fixture['env']['expectedPeriodNumber'],
            period_start_prevhash=fixture['env']['periodStartHash'],
            parent_hash=fixture['env']['previousHash'],
            coinbase=fixture['env']['currentCoinbase'],
            number=fixture['env']['currentNumber'],
        )

    chaindb = ChainDB(
        get_db_backend(),
        account_state_class=account_state_class,
        trie_class=trie_class
    )
    vm = fixture_vm_class(header=header, chaindb=chaindb)

    vm_state = vm.state
    with vm_state.mutable_state_db() as state_db:
        state_db.apply_state_dict(fixture['pre'])
    # Update state_root manually
    vm.block.header.state_root = vm_state.state_root
    if 'secretKey' in fixture['transaction']:
        unsigned_transaction = vm.create_unsigned_transaction(
            nonce=fixture['transaction']['nonce'],
            gas_price=fixture['transaction']['gasPrice'],
            gas=fixture['transaction']['gasLimit'],
            to=fixture['transaction']['to'],
            value=fixture['transaction']['value'],
            data=fixture['transaction']['data'],
        )
        private_key = keys.PrivateKey(fixture['transaction']['secretKey'])
        transaction = unsigned_transaction.as_signed_transaction(private_key=private_key)
    elif 'vrs' in fixture['transaction']:
        v, r, s = (
            fixture['transaction']['v'],
            fixture['transaction']['r'],
            fixture['transaction']['s'],
        )
        transaction = vm.create_transaction(
            nonce=fixture['transaction']['nonce'],
            gas_price=fixture['transaction']['gasPrice'],
            gas=fixture['transaction']['gasLimit'],
            to=fixture['transaction']['to'],
            value=fixture['transaction']['value'],
            data=fixture['transaction']['data'],
            v=v,
            r=r,
            s=s,
        )
    else:
        # sharding transaction
        transaction = vm.create_transaction(
            chain_id=fixture['transaction']['chainID'],
            shard_id=fixture['transaction']['shardID'],
            to=fixture['transaction']['to'],
            data=fixture['transaction']['data'],
            gas=fixture['transaction']['gasLimit'],
            gas_price=fixture['transaction']['gasPrice'],
            access_list=fixture['transaction']['accessList'],
            code=fixture['transaction']['code'],
        )

    try:
        computation, _ = vm.apply_transaction(transaction)
    except ValidationError as err:
        transaction_error = err
        LOGGER.warn("Got transaction error", exc_info=True)
    else:
        transaction_error = False

    if not transaction_error:
        log_entries = computation.get_log_entries()
        actual_logs_hash = hash_log_entries(log_entries)
        if 'logs' in fixture['post']:
            expected_logs_hash = fixture['post']['logs']
            assert expected_logs_hash == actual_logs_hash
        elif log_entries:
            raise AssertionError("Got log {0} entries. hash:{1}".format(
                len(log_entries),
                actual_logs_hash,
            ))

        if 'out' in fixture:
            expected_output = fixture['out']
            if isinstance(expected_output, int):
                assert len(computation.output) == expected_output
            else:
                assert computation.output == expected_output 
    print(vm.block.header.state_root == fixture['post']['hash'])
    
test_state_fixtures(normalize_fixture, TangerineWhistleVMForTesting)

b'~\x7f\xf8.\x1c3\xff\xbe\xb3\xde\xdbg\x94\x98\xc9qo\x83\xc6\x97\xf4\x97\xd7Z\xcaFz\xe96\xb8\xaeE'
b'~\x7f\xf8.\x1c3\xff\xbe\xb3\xde\xdbg\x94\x98\xc9qo\x83\xc6\x97\xf4\x97\xd7Z\xcaFz\xe96\xb8\xaeE'
True
